In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
transfers_csv = "../Resources/transfers.csv"
transfers_df = pd.read_csv(transfers_csv)
transfers_df.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [3]:
transfers_df = transfers_df.rename(columns={"Name": "name", "Position": "position", "Age": "age", "Team_from": "team_from",
                             "League_to": "league_to","League_from": "league_from", "Team_to":"team_to", "Season":"season",
                             "Market_value":"market_value", "Transfer_fee":"transfer_fee"})
transfers_df['market_value'] = transfers_df['market_value'] .fillna(0)
# transfers_df.set_index("name", inplace=True)
transfers_df

,name,position,age,team_from,league_from,team_to,league_to,season,market_value,transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,0.0,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,0.0,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,0.0,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,0.0,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,0.0,34500000
...,...,...,...,...,...,...,...,...,...,...
4695,Jasmin Kurtic,Attacking Midfield,29,Atalanta,Serie A,SPAL,Serie A,2018-2019,5000000.0,4800000
4696,Tchê Tchê,Central Midfield,25,Palmeiras,Série A,Dynamo Kyiv,Premier Liga,2018-2019,3000000.0,4800000
4697,Silvan Widmer,Right-Back,25,Udinese Calcio,Serie A,FC Basel,Super League,2018-2019,8500000.0,4500000
4698,Yuya Osako,Second Striker,28,1. FC Köln,2.Bundesliga,Werder Bremen,1.Bundesliga,2018-2019,4500000.0,4500000


In [4]:
locations_csv = "../Resources/locations.csv"
locations_df = pd.read_csv(locations_csv)
locations_df = locations_df.rename(columns={"Country":"country","Code": "country_code"})
# locations_df = locations_df.drop(columns=['Unnamed: 4', 'Unnamed: 5'])
locations_df = locations_df.dropna()
locations_df.set_index("country", inplace=True)
locations_df

,country_code,latitude,longitude
country,,,
Afghanistan,AFG,33.939110,67.709953
Albania,ALB,41.153332,20.168331
Algeria,DZA,28.033886,1.659626
American Samoa,ASM,-14.270972,-170.132217
Andorra,AND,42.546245,1.601554
...,...,...,...
Wallis Islands and Futuna,WLF,-13.768752,-177.156097
Western Sahara,ESH,24.215527,-12.885834
Yemen,YEM,15.552727,48.516388


In [5]:
leagues_csv = "../Resources/league_countries.csv"
leagues_df = pd.read_csv(leagues_csv)
leagues_df = leagues_df.rename(columns={"League Name": 'league_name', "Country":"country"})
leagues_df.set_index("league_name", inplace=True)
leagues_df

,country
league_name,
LaLiga,Spain
Serie A,Italy
Premier League,England
Ligue 1,France
LaLiga2,Spain
...,...
Série A,Italy
2ª B - Grupo III,Spain
Série B,Italy


In [6]:
connection_string = "postgres:taunus48@localhost:5432/football_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
engine.table_names()

['league_countries', 'transfers', 'locations']

In [8]:
locations_df.to_sql(name='locations', con=engine, if_exists='append', index=True)

In [9]:
leagues_df.to_sql(name='league_countries', con=engine, if_exists='append', index=True)


In [10]:
transfers_df.to_sql(name='transfers', con=engine, if_exists='append', index=True)

In [19]:
football_df = pd.read_sql_query('select * from transfers\
                join league_countries on league_countries.league_name = transfers.league_from\
                  join locations on locations.country = league_countries.country', con=engine)
football_df

,index,name,position,age,team_from,league_from,team_to,league_to,season,market_value,transfer_fee,league_name,country,country,country_code,latitude,longitude
0,0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,0,60000000,LaLiga,Spain,Spain,ESP,40.463667,-3.7492199999999998
1,1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,0,56810000,Serie A,Italy,Italy,ITA,41.87194,12.56738
2,2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,0,40000000,Premier League,England,England,GBR,55.378051,-3.435973
3,3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,0,36150000,Serie A,Italy,Italy,ITA,41.87194,12.56738
4,4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,0,34500000,LaLiga,Spain,Spain,ESP,40.463667,-3.7492199999999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4695,4695,Jasmin Kurtic,Attacking Midfield,29,Atalanta,Serie A,SPAL,Serie A,2018-2019,5000000,4800000,Serie A,Italy,Italy,ITA,41.87194,12.56738
4696,4696,Tchê Tchê,Central Midfield,25,Palmeiras,Série A,Dynamo Kyiv,Premier Liga,2018-2019,3000000,4800000,Série A,Italy,Italy,ITA,41.87194,12.56738
4697,4697,Silvan Widmer,Right-Back,25,Udinese Calcio,Serie A,FC Basel,Super League,2018-2019,8500000,4500000,Serie A,Italy,Italy,ITA,41.87194,12.56738
4698,4698,Yuya Osako,Second Striker,28,1. FC Köln,2.Bundesliga,Werder Bremen,1.Bundesliga,2018-2019,4500000,4500000,2.Bundesliga,Germany,Germany,DEU,51.165690999999995,10.451526


In [22]:
import json

# foot_array = []

# for index in football_db:
#     foot_dict = {}
#     foot_dict['name'] = index
# #     foot_dict['season'] = season
# #     foot_dict['market_value'] = market_value
# #     foot_dict['transfer_fee'] = transfer_fee
# #     foot_dict['country'] = country
# #     foot_dict['locations'] = locations
# foot_array.append(foot_dict)
# foot_array
data = football_df.to_json(r'futbol.json',orient='table')
print(data)

None


In [21]:
# json_json = open("futbol.json", "w")
# json.dump(data, json_json, indent=6)

In [ ]:
Base.classes.keys()

In [ ]:
)
    